In [ ]:
import json
from elasticsearch import Elasticsearch
from pyhive import hive
import pandas as pd
from sentence_transformers import SentenceTransformer

# Consulta hive

In [3]:
# Conexión a HiveServer2
conn = hive.Connection(host='hiveserver2', port=10000, username='hugo', database='default')

# Crear un cursor
cursor = conn.cursor()
cursor = conn.cursor()

# Ejecutar una consulta
cursor.execute("""SELECT DISTINCT d.driverid, d.forename, d.surname
FROM races r
JOIN laps l ON r.raceid = l.raceid
JOIN drivers d ON l.driverid = d.driverid
WHERE r.name = 'Malaysian Grand Prix'
  AND l.lap = (SELECT MAX(lap) FROM laps WHERE raceid = r.raceid)
  AND l.position = 1""")

# Obtener los resultados
hive_results = pd.DataFrame(cursor.fetchall(), columns=['driverid', 'forename', 'surname'])

display(hive_results)

# Cerrar la conexión
cursor.close()
conn.close()

,driverid,forename,surname
0,1,Lewis,Hamilton
1,4,Fernando,Alonso
2,8,Kimi,Räikkönen
3,18,Jenson,Button
4,20,Sebastian,Vettel
5,21,Giancarlo,Fisichella
6,23,Ralf,Schumacher
7,30,Michael,Schumacher
8,56,Eddie,Irvine
9,817,Daniel,Ricciardo


# Consulta elasticsearch

In [6]:
es = Elasticsearch( "http://elasticsearch:9200")

index_name = 'drivers_descriptions'

model = SentenceTransformer('intfloat/multilingual-e5-large')

In [10]:
query_sentence = "query: pilotos que corrieron en le mans y en indianapolis"
query_vector = model.encode([query_sentence])[0]

parameters = {
    "field":"wikipedia_description_vector",
    "query_vector": query_vector,
    "k": 100,
    "num_candidates":100
}

res = es.search(index=index_name, knn=parameters, size=100)
es_results = pd.DataFrame(
    [{"driverid": x["_source"]["driverId"], "wikipedia_description": x["_source"]["wikipedia_description"]}
     for x in res['hits']['hits']])
display(es_results)

,driverid,wikipedia_description
0,296,François Migault (4 December 1944 – 29 January...
1,7,Sébastien Olivier Bourdais[1] (French pronunci...
2,513,"Eddie Johnson (February 10, 1919 – June 30, 19..."
3,212,Héctor Alonso Rebaque (born 5 February 1956)[1...
4,615,"Fred Levon Agabashian[1][2] (August 21, 1913 –..."
...,...,...
95,831,Luiz Felipe de Oliveira Nasr (born 21 August 1...
96,66,Allan McNish (born 29 December 1969) is a Brit...
97,197,Jean-Pierre Jacques Jarier[1] (born 10 July 19...
98,627,Louis Claude Rosier (5 November 1905 in Chapde...


# Join dataframe

In [9]:
# Realizar el join por driverid
combined_results = pd.merge(hive_results, es_results, on="driverid", how="inner")

display(combined_results)


,driverid,forename,surname,wikipedia_description
0,4,Fernando,Alonso,Fernando Alonso Díaz (Spanish pronunciation: [...
1,56,Eddie,Irvine,"Edmund ""Eddie"" Irvine Jr. (/ˈɜːrvaɪn/; born 10..."
